In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#print(loaded)
!pip install mne
!pip install mne[hdf5]
!pip install madmom
!pip install ffmpeg
!pip install mir_eval

#!conda activate mne

     |████████████████████████████████| 7.5 MB 12.8 MB/s 
     |████████████████████████████████| 20.0 MB 10.2 MB/s 
     |████████████████████████████████| 51 kB 6.8 MB/s 
  Created wheel for madmom: filename=madmom-0.16.1-cp37-cp37m-linux_x86_64.whl size=20935879 sha256=a728fad56287cac6bd238df5f0299fb9069dd1909daebc227051e79b8a659d20
  Stored in directory: /root/.cache/pip/wheels/af/90/61/393ceef814b55b12d1b59b5ed3a2b2a3457a55d39b7363b975
Successfully built madmom
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6084 sha256=d2a9884c30eb3cfe964f6821cf9ed7765cf1957d41f4c915d9c3d0dfd3740245
  Stored in directory: /root/.cache/pip/wheels/64/80/6e/caa3e16deb0267c3cbfd36862058a724144e19fdb9eb03af0f
Successfully built ffmpeg
     |████████████████████████████████| 90 kB 6.2 MB/s 
  Created wheel for mir-eval: filename=mir_eval-0.7-py3-none-any.whl size=100721 sha256=277e0c047406caddbad1695aaa44f1b358d955d527304876da16b88a1c766a97
  Stored in directory: /root/.cache/pip/

In [ ]:
# from google.colab import drive
# drive.mount("/content/drive")
# %cd '/content/drive/MyDrive/2022 Spring/11-785 Introduction to Deep Learning/Project'

In [ ]:
import matplotlib.pyplot as plt

from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import DetCurveDisplay, RocCurveDisplay
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

In [ ]:
from scipy import io
import os
import numpy as np
from sklearn.metrics import roc_auc_score
import mne
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import f1_score

# from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import torchaudio
# import wandb 
import madmom
import madmom.features.beats
import madmom.features.onsets
import mir_eval
import ffmpeg
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import gc
import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime


In [ ]:
data_folder='/content/drive/MyDrive/11785 Project/data'
#data_folder = '/content/drive/MyDrive/11785-IDL/11785 Project/data'


#Restructuring the EEG folders

In [ ]:
timestamps = np.load(data_folder+'/ground_truths/timestamps.npy',allow_pickle=True)
len(timestamps)

10

In [ ]:
one_hot = np.load(data_folder+'/ground_truths/one_hot.npy',allow_pickle=True)
one_hot.shape

(10, 37500)

In [ ]:
data = io.loadmat(data_folder+'/users_mat/user0_restructured.mat')

In [ ]:
EEG = data['data']
EEG.shape

(10, 125, 37500)

In [ ]:
from scipy import io
loaded = io.loadmat(data_folder +'/users_mat/user0_restructured.mat',  struct_as_record=True)

# loaded = io.loadmat('/content/song21_Imputed.mat',  struct_as_record=True)
print(loaded.keys())
#dict_keys(['__header__', '__version__', '__globals__', 'mrk', 'cnt', 'nfo'])

dict_keys(['__header__', '__version__', '__globals__', 'data', 'user'])


In [ ]:
class UserDataset(torch.utils.data.Dataset):
     def __init__(self, data_path, partition= "train", user_id = "0"): # You can use partition to specify train or dev

        self.X_dir = data_path + "/users_mat/" # TODO: get mfcc directory path
        self.Y_dir = data_path + "/ground_truths/" # TODO: get transcript path

        self.X_files = os.listdir(self.X_dir)# TODO: list files in the mfcc directory
        print(self.X_files)
        if partition == 'train':
            self.X_files = [f for f in self.X_files if user_id not in f]
            val = 19
        else:
            self.X_files = [f for f in self.X_files if user_id in f]
            val = 1
        
        self.Y_onehot = np.load(data_path+'/ground_truths/one_hot.npy')
        self.Y_timestamps = np.load(data_path+'/ground_truths/timestamps.npy',allow_pickle=True)
        # self.Y_files = os.listdir(self.Y_dir)# TODO: list files in the transcript directory
        self.XXX = []
        self.YYY = []
  

        for x_file in self.X_files:
            Xtemp =  io.loadmat(data_folder+'/users_mat/'+x_file)
            data = Xtemp['data']
            data = mne.filter.filter_data(data,sfreq=100,l_freq=0.1,h_freq = 40)
            for i in data:
              #print("1",i.shape)
              temp = np.reshape(i,(300,125,125))
              #print("2",temp.shape)
              split_temp = np.split(temp,300)
              #print(len(split_temp))
              #print("3",split_temp[0].shape)
              self.XXX.extend(split_temp)
            
            

            for i in self.Y_onehot:
                temp = np.split(i, 300)
                #print(len(temp))
                self.YYY.extend(temp)
              
        

        print(len(self.YYY))
        print(len(self.XXX))


        pass

     def __len__(self):
        return len(self.X_files)

     def __getitem__(self, ind):

        return self.XXX[ind], self.YYY[ind]
    



In [ ]:
# print(len(users))

In [ ]:
# users = UserDataset(data_folder, partition= "train", user_id = "user0")
# # print(users[0])

In [ ]:
class UserRNNValidDataset(torch.utils.data.Dataset):
    def __init__(self, data_path, user_id = "0"): # You can use partition to specify train or dev
        self.X_dir = data_path + "/users_mat/" # TODO: get mfcc directory path
        self.Y_dir = data_path + "/ground_truths/" # TODO: get transcript path

        self.X_files = os.listdir(self.X_dir)# TODO: list files in the mfcc directory
        self.X_files = [f for f in self.X_files if user_id in f]
        val = 1
        print(self.X_files)
        
        self.Y_onehot = np.load(data_path+'/ground_truths/one_hot.npy')
        self.Y_timestamps = np.load(data_path+'/ground_truths/timestamps.npy',allow_pickle=True)
        # self.Y_files = os.listdir(self.Y_dir)# TODO: list files in the transcript directory
        self.XXX = []
        self.YYY = []
        
        for x_file in tqdm(self.X_files):
            Xtemp =  io.loadmat(data_folder+'/users_mat/'+x_file)
            data = Xtemp['data']
            data = mne.filter.filter_data(data,sfreq=100,l_freq=0.1,h_freq = 40)
            for i in data:
                x = i.T
                # temp = np.reshape(i,(300,125,125))
                # split_temp = np.split(temp,300)
                self.XXX.append(x)


            for i in self.Y_onehot:
                # y = np.reshape(i,(300,125))
                # temp = np.split(i, 300)
                self.YYY.append(i)

                # self.YYY.extend(temp)

        assert(len(self.YYY) == len(self.XXX))

    def __len__(self):
        return len(self.XXX)

    def __getitem__(self, ind):
        return self.XXX[ind], self.YYY[ind]
    
    def collate_fn(batch):
        batch_x = [torch.from_numpy(x) for x,y in batch]
        batch_y = [torch.from_numpy(y) for x,y in batch]

        return torch.stack(batch_x,dim=0).squeeze(dim=0),torch.stack(batch_y,dim=0).squeeze(dim=0)



In [ ]:
class UserRNNDataset(torch.utils.data.Dataset):
    def __init__(self, data_path, partition= "train", user_id = "0",context=0): # You can use partition to specify train or dev
        self.X_dir = data_path + "/users_mat/" # TODO: get mfcc directory path
        self.Y_dir = data_path + "/ground_truths/" # TODO: get transcript path
        self.context = context
        self.X_files = os.listdir(self.X_dir)# TODO: list files in the mfcc directory
        print(user_id)
        if partition == 'train':
            self.X_files = [f for f in self.X_files if user_id not in f]
            val = 19
        else:
            self.X_files = [f for f in self.X_files if user_id in f]
            val = 1
        print(self.X_files)
        
        self.Y_onehot = np.load(data_path+'/ground_truths/one_hot.npy')
        self.Y_timestamps = np.load(data_path+'/ground_truths/timestamps.npy',allow_pickle=True)
        # self.Y_files = os.listdir(self.Y_dir)# TODO: list files in the transcript directory
        XXX = []
        self.YYY = []
        
        for x_file in self.X_files:
            Xtemp =  io.loadmat(data_folder+'/users_mat/'+x_file)
            data = Xtemp['data']
            data = mne.filter.filter_data(data,sfreq=100,l_freq=0.1,h_freq = 40)
            for i in data:
                x = i.T
                temp = np.reshape(i,(300,125,125))
                split_temp = np.split(temp,300)
                XXX.extend(split_temp)
            
            del Xtemp,data

            for i in self.Y_onehot:
                y = np.reshape(i,(300,125))
                temp = np.split(i, 300)
                self.YYY.extend(temp)

        self.XX =np.array(XXX)
        # print(self.XX.shape)
        if context!=0:
            self.add_blocks = context//125 + 1
            self.XX = np.pad(self.XX,((self.add_blocks,self.add_blocks),(0,0),(0,0),(0,0)))
            # print("post pad",self.XX.shape)
        
        # assert(len(self.YYY) == len(self.XXX))

    def __len__(self):
        return len(self.YYY)

    def __getitem__(self, ind):
        if self.context!=0:
            extracted = self.XX[ind:ind+(self.add_blocks*2)+1,0]
            pre_x = extracted[0,-self.context%125:,:]
            post_x = extracted[-1,:self.context%125,:]
            # print("pre",pre_x.shape)
            # print("post",post_x.shape)
            middle_x = extracted[1:-1,:,:].reshape((-1,125))
            # print("middle",middle_x.shape)
            final_x = np.vstack((pre_x,middle_x,post_x))
            # print('final_x',final_x.shape)
            return final_x,self.YYY[ind]
        else:
            return self.XXX[ind], self.YYY[ind]
    
    def collate_fn(batch):
        batch_x = [torch.from_numpy(x) for x,y in batch]
        batch_y = [torch.from_numpy(y) for x,y in batch]

        return torch.stack(batch_x,dim=0).squeeze(),torch.stack(batch_y,dim=0).squeeze()



In [ ]:
user_id=0
context = 5
# train_dataset = UserRNNDataset(data_folder,'train','user'+str(user_id),context = context)
# val_dataset = UserRNNDataset(data_folder,partition="valid",user_id='user'+str(user_id),context = context)
# train_loader =torch.utils.data.DataLoader(train_dataset,batch_size=32,collate_fn = UserRNNDataset.collate_fn,shuffle=True,num_workers=4)
# val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=32,collate_fn = UserRNNDataset.collate_fn,shuffle=False,num_workers=4)


In [ ]:
# class Baseline(torch.nn.Module):
#     def __init__(self,context=0):
#         super(Baseline, self).__init__()
#         # self.flatten = nn.Flatten()
#         self.context_embedding = nn.Conv1d(in_channels=125, out_channels=125,kernel_size = 2*context+1,padding="valid",stride=1,groups = 125)
#         self.rnn = nn.GRU(input_size = 125,hidden_size=64,num_layers=2,batch_first=True)

#         self.layers = nn.Sequential(
#             nn.Linear(64, 1), # try with 125 output
#         )

#     def forward(self, x):
#         # x = torch.flatten(x,start_dim=-2)
#         # print('forwaard')
#         trans_x = torch.permute(x,(0,2,1))
#         # print("perm inp",trans_x.shape)
#         x_out = self.context_embedding(trans_x)
#         # print("out of conv",x_out.shape)
#         x = torch.permute(x_out,(0,2,1))
#         # print("conv op permute",x.shape)
#         out,h_n =self.rnn(x)
#         x = self.layers(out)
#         # print("")
#         return x


In [ ]:
# class Baseline(torch.nn.Module):
#     def __init__(self,context=0):
#         super(Baseline, self).__init__()
#         # self.flatten = nn.Flatten()
#         self.context_embedding = nn.Sequential(
#             nn.Conv1d(in_channels=125, out_channels=125,kernel_size = 2*context+1,padding="valid",stride=1,groups = 125),
#             nn.BatchNorm1d(125),
#             nn.LeakyReLU(),)
#         self.rnn = nn.GRU(input_size = 125,hidden_size=64,num_layers=2,batch_first=True)

#         self.layers = nn.Sequential(
#             nn.Linear(64, 64),
#             nn.BatchNorm1d(64),
#             nn.LeakyReLU(),
#             nn.Dropout(0.35),
#             nn.Linear(64, 1), # try with 125 output
#         )

#     def forward(self, x):
#         trans_x = torch.permute(x,(0,2,1))
#         x_out = self.context_embedding(trans_x)
#         x = torch.permute(x_out,(0,2,1))
#         out,h_n =self.rnn(x)
#         x = self.layers(out)
#         # print("")
#         return x


In [ ]:
# class Baseline(torch.nn.Module):
#     def __init__(self,context=0):
#         super(Baseline, self).__init__()
#         # self.flatten = nn.Flatten()
#         #self.context_embedding = nn.Conv1d(in_channels=125, out_channels=125,kernel_size = 2*context+1,padding="valid",stride=1,groups = 125)
        
#         self.context_embedding = nn.Sequential(
#             nn.Conv1d(in_channels=125,out_channels=125,kernel_size=2*context+1,stride=1, padding="valid",groups = 125),
#         )
        
#         #self.rnn = nn.GRU(input_size = 125,hidden_size=64,num_layers=2,batch_first=True)
#         self.lstm = nn.LSTM(125,125, 3, batch_first = True, dropout = 0.4, bidirectional=True)

#         #self.layer_norm = nn.LayerNorm(125)

#         self.layers = nn.Sequential(
#             nn.Linear(250, 2056),
#             nn.LeakyReLU(),
#             nn.Dropout(0.3),
#             nn.Linear(2056, 1), # try with 125 output
#         )

#     def forward(self, x):
#         trans_x = torch.permute(x,(0,2,1))
#         x_out = self.context_embedding(trans_x)
#         x = torch.permute(x_out,(0,2,1))
#         out,h_n =self.lstm(x)
#         #out = self.layer_norm(out)
#         x = self.layers(out)
#         # print("")
#         return x


In [ ]:
# class Baseline(torch.nn.Module):
#     def __init__(self,context=0):
#         super(Baseline, self).__init__()
#         # self.flatten = nn.Flatten()
#         self.context_embedding = nn.Conv1d(in_channels=125, out_channels=125,kernel_size = 2*context+1,padding="valid",stride=1,groups = 125)
#         #self.rnn = nn.GRU(input_size = 125,hidden_size=64,num_layers=2,batch_first=True)
#         self.lstm = nn.LSTM(125,125, 3, batch_first = True, dropout = 0.4, bidirectional=True)

#         self.layers = nn.Sequential(
#             nn.Linear(250, 1028),
#             nn.LeakyReLU(),
#             nn.Dropout(0.4),
#             nn.Linear(1028, 4096),
#             nn.LeakyReLU(),
#             nn.Dropout(0.4),
#             nn.Linear(4096, 1), # try with 125 output
#         )

#     def forward(self, x):
#         trans_x = torch.permute(x,(0,2,1))
#         x_out = self.context_embedding(trans_x)
#         x = torch.permute(x_out,(0,2,1))
#         out,h_n =self.lstm(x)
#         x = self.layers(out)
#         # print("")
#         return x

In [ ]:
class Baseline(torch.nn.Module):
    def __init__(self,context=0):
        super(Baseline, self).__init__()
        # self.flatten = nn.Flatten()
        self.context_embedding = nn.Conv1d(in_channels=125, out_channels=125,kernel_size = 2*context+1,padding="valid",stride=1,groups = 125)
        #self.rnn = nn.GRU(input_size = 125,hidden_size=64,num_layers=2,batch_first=True)
        self.lstm = nn.LSTM(125,125, 3, batch_first = True, dropout = 0.4, bidirectional=True)

        self.layers = nn.Sequential(
            nn.Linear(250, 1028),
            nn.LeakyReLU(),
            nn.Dropout(0.3),
            nn.Linear(1028, 2048),
            nn.LeakyReLU(),
            nn.Dropout(0.3),
            nn.Linear(2048, 1), # try with 125 output
        )

    def forward(self, x):
        trans_x = torch.permute(x,(0,2,1))
        x_out = self.context_embedding(trans_x)
        x = torch.permute(x_out,(0,2,1))
        out,h_n =self.lstm(x)
        x = self.layers(out)
        # print("")
        return x

In [ ]:
# model(x.float()).shape
from sklearn.metrics import det_curve
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import RocCurveDisplay

In [ ]:
def validate(model,val_loader,criterion):
    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, leave=False, position=0, desc='Validation')

    f1=[]

    recall=[]
    precision=[]
    total_val_loss = 0
    ground_truth=np.empty((1,0))
    predictions = np.empty((1,0))
    # print("Validation")
    total_raw_score=0
    for idx,(x,y) in tqdm(enumerate(val_loader)):
    # for idx,(x,y) in enumerate(val_loader):

        ground_truth = np.append(ground_truth,y.reshape(1,-1))
        x = x.float().cuda()
        y = y.float().cuda()

        with torch.no_grad():
            pd_one_hot = model(x)
            predictions = np.append(predictions,nn.Sigmoid()(pd_one_hot).squeeze().cpu().detach().numpy().reshape(1,-1))
            val_loss = criterion(pd_one_hot.squeeze(),y)
            y_np = y.cpu().numpy()
            pred_np = nn.Sigmoid()(pd_one_hot).squeeze().cpu().detach().numpy()
            total_raw_score += roc_auc_score(y_np.flatten(),pred_np.flatten())
            #fpr, fnr, thresholds = det_curve(y_np.flatten(),pred_np.flatten())
            # pd_one_hot = nn.Sigmoid()(pd_one_hot)
            # pd_one_hot = torch.flatten(pd_one_hot)
            # pd_one_hot = pd_one_hot.cpu().numpy()
        total_val_loss +=val_loss
        # out = madmom.features.onsets.peak_picking(pd_one_hot,0.5)
        # predicted_ts = out/125
        # if len(predicted_ts)==0:
        #     F,P,R =0.0,0.0,0.0
        # else:
        #     F,P,R = mir_eval.onset.f_measure(gt_ts,predicted_ts,window=1)
        # f1.append(F)
        # precision.append(P)
        # recall.append(R)
        #print(pd_one_hot, predictions)
        #print(y_np, pred_np)
        batch_bar.set_postfix(roc_score="{:.04f}".format(float(total_raw_score / (idx + 1))),loss="{:.04f}".format(float(total_val_loss / (idx + 1))), lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        batch_bar.update()
        #RocCurveDisplay.from_predictions(y_np,pred_np)
        del x,y,pd_one_hot,val_loss
        torch.cuda.empty_cache()
    # print(predictions.shape)
    batch_bar.close()

    for idx in range(10):
        print("")
        gt_ts = val_loader.dataset.Y_timestamps[idx]
        song_pred = predictions[idx*37500:(idx+1)*37500]

        # print(song_pred.shape,gt_ts.shape)
        #print(len(gt_ts))
        out = madmom.features.onsets.peak_picking(song_pred,0.055)
        predicted_ts = out/125
        #print(len(predicted_ts))
        if len(predicted_ts)==0:
            F,P,R =0.0,0.0,0.0
        else:
            F,P,R = mir_eval.onset.f_measure(gt_ts,predicted_ts,window=0.1)
        f1.append(F)
        precision.append(P)
        recall.append(R)

    
    plt.show()


    model.train()
    return f1,precision,recall,total_val_loss/len(val_loader)


In [ ]:
epoch = 13
scores = {"f":[],'p':[],'r':[]}
batch_size = 64

In [ ]:
# del train_dataset,val_dataset,train_loader,val_loader
# del model,criterion
# del optimizer

In [ ]:
def train(model,epoch,train_loader,val_loader,criterion,optimizer, scheduler):
    for e in range(epoch):
        print("Epoch ",e)
        batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Training')
        total_loss=0
        for idx,data in enumerate(train_loader):
            model.train()
            optimizer.zero_grad()
            x=data[0].float().cuda()
            y=data[1].float().cuda()
            out = model(x)
            out=out.squeeze()
            loss = criterion(out,y)
            loss.backward()
            optimizer.step()
            total_loss+=loss

            batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (idx + 1))), lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
            batch_bar.update()

            del data,x,y,out,loss
            torch.cuda.empty_cache()
        batch_bar.close()

        f,p,r,val_loss  = validate(model,val_loader,criterion)
        

        print("Epoch {}/{}: Train Loss {:.04f}, Learning Rate {:.04f}, Validation Loss {:.04f}, F1 Score {:.04f}, Precision {:.04f}, Recall {:.04f}".format(
            e,
            epoch,
            float(total_loss / (len(train_loader))),
            float(optimizer.param_groups[0]['lr']),val_loss,np.mean(f),np.mean(p),np.mean(r)))
        
        scheduler.step(val_loss)
    return np.mean(f),np.mean(p),np.mean(r)

In [ ]:
import gc
valid_ids = [0,5,10,15]

pos_weight = torch.tensor([80/125]).cuda()
pos_weight = pos_weight.cuda()

batch_size = 32

# for user_id in valid_ids:
for user_id in valid_ids:
    print('user_id', user_id)
    gc.collect()
    context = 60
    train_dataset = UserRNNDataset(data_folder,'train','user'+str(user_id),context = context)
    val_dataset = UserRNNDataset(data_folder,partition="valid",user_id='user'+str(user_id),context = context)
    train_loader =torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,collate_fn = UserRNNDataset.collate_fn,shuffle=True,num_workers=4)
    val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=batch_size,collate_fn = UserRNNDataset.collate_fn,shuffle=False,num_workers=4)
    model = Baseline(context=context).cuda()
    #model = Network(context=context).cuda()
    #criterion = nn.BCEWithLogitsLoss()
    criterion = nn.BCEWithLogitsLoss(pos_weight = pos_weight)
    optimizer = torch.optim.Adam(model.parameters(),lr = 1e-3)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, threshold = 0.1, factor = 0.3, patience = 2, mode = 'min')


    f,p,r = train(model,epoch,train_loader,val_loader,criterion,optimizer, scheduler)
    scores['f'].append(f)
    scores['p'].append(p)
    scores['r'].append(r)

    del train_dataset,val_dataset,train_loader,val_loader,model,criterion,optimizer
    torch.cuda.empty_cache()
    gc.collect()


user_id 0


user0
['user1_restructured.mat', 'user2_restructured.mat', 'user3_restructured.mat', 'user4_restructured.mat', 'user5_restructured.mat', 'user6_restructured.mat', 'user7_restructured.mat', 'user8_restructured.mat', 'user9_restructured.mat', 'user10_restructured.mat', 'user11_restructured.mat', 'user12_restructured.mat', 'user13_restructured.mat', 'user14_restructured.mat', 'user15_restructured.mat', 'user16_restructured.mat', 'user17_restructured.mat', 'user18_restructured.mat', 'user19_restructured.mat']
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter 

Validation:  97%|█████████▋| 73/75 [00:04<00:00, 24.92it/s, loss=0.1249, lr=0.0010, roc_score=0.5099]
75it [00:03, 21.51it/s]












Epoch 0/13: Train Loss 0.1267, Learning Rate 0.0010, Validation Loss 0.1252, F1 Score 0.0000, Precision 0.0000, Recall 0.0000
Epoch  1


Validation:  97%|█████████▋| 73/75 [00:04<00:00, 24.97it/s, loss=0.1252, lr=0.0010, roc_score=0.5123]
75it [00:03, 21.70it/s]


...











Epoch 10/13: Train Loss 0.1127, Learning Rate 0.0000, Validation Loss 0.1228, F1 Score 0.6027, Precision 0.7510, Recall 0.5149
Epoch  11


Validation:  97%|█████████▋| 73/75 [00:04<00:00, 23.70it/s, loss=0.1226, lr=0.0000, roc_score=0.6147]
75it [00:03, 21.10it/s]












Epoch 11/13: Train Loss 0.1125, Learning Rate 0.0000, Validation Loss 0.1229, F1 Score 0.5775, Precision 0.7550, Recall 0.4789
Epoch  12


Validation:  97%|█████████▋| 73/75 [00:04<00:00, 24.22it/s, loss=0.1226, lr=0.0000, roc_score=0.6154]
75it [00:03, 21.16it/s]
                                                                                                     











Epoch 12/13: Train Loss 0.1124, Learning Rate 0.0000, Validation Loss 0.1228, F1 Score 0.6036, Precision 0.7526, Recall 0.5155


In [ ]:
print("Final F1 score : ", sum(scores['f'][:])/4)
print("Final precision score : ",sum(scores['p'][:])/4)
print("Final recall score : ",sum(scores['r'][:])/4)
#print(scores)

Final F1 score :  0.6100101703621045
Final precision score :  0.7502793599237499
Final recall score :  0.5287019845460492
{'f': [0.64673023500123, 0.6219661859768031, 0.5677899588771755, 0.6035543015932093], 'p': [0.7345130370298107, 0.7528168761273755, 0.7611383349003547, 0.752649191637459], 'r': [0.5934019408563976, 0.5432566214139163, 0.46261962614499674, 0.5155297497688861]}
